In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install folium

In [ ]:
!pip install pulp

In [ ]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

In [ ]:
# 거리 계산 함수
def calculate_distance(x1, y1, x2, y2):
    return geodesic((y1, x1), (y2, x2)).kilometers

# 수요지와 후보지 데이터 불러오기
demand = pd.read_csv("/content/drive/MyDrive/주제1/data.csv")
candidate = pd.read_csv("/content/drive/MyDrive/주제1/target.csv")

# 이상치 제거
# 삭제할 인덱스 리스트
indexes_to_remove = [0,3,4,6,10,14,18]
candidate = candidate.drop(indexes_to_remove)
candidate = candidate.reset_index(drop=True)

# 후보지 개수 설정
num_candidates = 43

# 후보지 인구수 제한 설정
population_limit = 750

# P-Median 문제 정의
model = LpProblem("Unbalanced P-Median", sense=LpMinimize)

# 후보지 선택 변수
candidates = LpVariable.dicts('Candidate', [j for j in range(num_candidates)], lowBound=0, cat='Binary')

# 각 수요지를 각 후보지에 할당하는 이진 변수
num_demands = len(demand)
assignments = LpVariable.dicts('Assignment', [(i, j) for i in range(num_demands) for j in range(num_candidates)], lowBound=0, cat='Binary')

# 목적 함수 정의: 이동 거리 최소화
model += lpSum(demand.loc[i, '초등학령인구'] *
    calculate_distance(demand.loc[i, 'X'], demand.loc[i, 'Y'], candidate.loc[j, 'X'], candidate.loc[j, 'Y'])
    * assignments[(i, j)] for i in range(num_demands) for j in range(num_candidates))

# 제약 조건
for i in range(num_demands):
    model += lpSum(assignments[(i, j)] for j in range(num_candidates)) == 1

# 인원수 제한
for j in range(num_candidates):
    model += lpSum(assignments[(i, j)] * demand.loc[i, '초등학령인구'] for i in range(num_demands)) <= population_limit * candidates[j]

# 문제 풀기
model.solve()

results = []
for i in range(num_demands):
    demand_id = demand.loc[i, '수요지 id']
    demand_population = demand.loc[i, '초등학령인구']
    nearest_candidates = []
    for j in range(num_candidates):
        if value(assignments[(i, j)]) == 1:
            nearest_candidates.append(candidate.loc[j, '학교 id'])
    results.append((demand_id, demand_population, nearest_candidates))

# 결과 출력
print("결과:")
for result in results:
    demand_id, demand_population, nearest_candidates = result
    print(f"수요지 id {demand_id} | 초등학령인구: {demand_population} | 할당된 후보지: {nearest_candidates}")

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


결과:
수요지 id C1 | 초등학령인구: 1 | 할당된 후보지: ['F43']
수요지 id C2 | 초등학령인구: 46 | 할당된 후보지: ['F45']
수요지 id C3 | 초등학령인구: 36 | 할당된 후보지: ['F47']
수요지 id C4 | 초등학령인구: 81 | 할당된 후보지: ['F45']
수요지 id C5 | 초등학령인구: 25 | 할당된 후보지: ['F45']
수요지 id C6 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C7 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C8 | 초등학령인구: 28 | 할당된 후보지: ['F47']
수요지 id C9 | 초등학령인구: 21 | 할당된 후보지: ['F47']
수요지 id C10 | 초등학령인구: 37 | 할당된 후보지: ['F45']
수요지 id C11 | 초등학령인구: 28 | 할당된 후보지: ['F47']
수요지 id C12 | 초등학령인구: 36 | 할당된 후보지: ['F47']
수요지 id C13 | 초등학령인구: 29 | 할당된 후보지: ['F3']
수요지 id C14 | 초등학령인구: 56 | 할당된 후보지: ['F45']
수요지 id C15 | 초등학령인구: 30 | 할당된 후보지: ['F45']
수요지 id C16 | 초등학령인구: 45 | 할당된 후보지: ['F45']
수요지 id C17 | 초등학령인구: 30 | 할당된 후보지: ['F47']
수요지 id C18 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C19 | 초등학령인구: 37 | 할당된 후보지: ['F47']
수요지 id C20 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C21 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C22 | 초등학령인구: 14 | 할당된 후보지: ['F47']
수요지 id C23 | 초등학령인구: 40 | 할당된 후보지: ['F3']
수요지 id C24 | 초등학령인구

In [ ]:
candidate_assignments = {j: [] for j in range(num_candidates)}

for i in range(num_demands):
    demand_id = demand.loc[i, '수요지 id']
    demand_population = demand.loc[i, '초등학령인구']
    nearest_candidates = []
    for j in range(num_candidates):
        if value(assignments[(i, j)]) == 1:
            nearest_candidates.append(candidate.loc[j, '학교 id'])
            candidate_assignments[j].append(demand_id)
    results.append((demand_id, demand_population, nearest_candidates))

print("결과:")
for j in range(num_candidates):
    candidate_id = candidate.loc[j, '학교 id']
    assigned_demand_count = len(candidate_assignments[j])
    assigned_population = sum(demand.loc[demand['수요지 id'].isin(candidate_assignments[j]), '초등학령인구'])
    print(f"후보지 id {candidate_id} | 할당된 수요지 개수: {assigned_demand_count} | 할당된 인구: {assigned_population}")

결과:
후보지 id F2 | 할당된 수요지 개수: 5 | 할당된 인구: 713
후보지 id F3 | 할당된 수요지 개수: 4 | 할당된 인구: 453
후보지 id F6 | 할당된 수요지 개수: 1 | 할당된 인구: 166
후보지 id F8 | 할당된 수요지 개수: 12 | 할당된 인구: 494
후보지 id F9 | 할당된 수요지 개수: 12 | 할당된 인구: 59
후보지 id F10 | 할당된 수요지 개수: 11 | 할당된 인구: 428
후보지 id F12 | 할당된 수요지 개수: 24 | 할당된 인구: 216
후보지 id F13 | 할당된 수요지 개수: 0 | 할당된 인구: 0
후보지 id F14 | 할당된 수요지 개수: 2 | 할당된 인구: 55
후보지 id F16 | 할당된 수요지 개수: 29 | 할당된 인구: 281
후보지 id F17 | 할당된 수요지 개수: 0 | 할당된 인구: 0
후보지 id F18 | 할당된 수요지 개수: 22 | 할당된 인구: 648
후보지 id F20 | 할당된 수요지 개수: 28 | 할당된 인구: 406
후보지 id F21 | 할당된 수요지 개수: 15 | 할당된 인구: 79
후보지 id F22 | 할당된 수요지 개수: 22 | 할당된 인구: 304
후보지 id F23 | 할당된 수요지 개수: 3 | 할당된 인구: 263
후보지 id F24 | 할당된 수요지 개수: 15 | 할당된 인구: 344
후보지 id F25 | 할당된 수요지 개수: 6 | 할당된 인구: 10
후보지 id F26 | 할당된 수요지 개수: 0 | 할당된 인구: 0
후보지 id F27 | 할당된 수요지 개수: 0 | 할당된 인구: 0
후보지 id F28 | 할당된 수요지 개수: 2 | 할당된 인구: 151
후보지 id F29 | 할당된 수요지 개수: 7 | 할당된 인구: 28
후보지 id F30 | 할당된 수요지 개수: 8 | 할당된 인구: 165
후보지 id F31 | 할당된 수요지 개수: 6 | 할당된 인구: 199
후보지 id F32 | 할당된 수요지

In [ ]:
distance_exceeded_results = []

# 결과 저장 및 거리 초과 여부 확인
for i in range(num_demands):
    demand_id = demand.loc[i, '수요지 id']
    demand_population = demand.loc[i, '초등학령인구']
    nearest_candidates = []
    for j in range(num_candidates):
        if value(assignments[(i, j)]) == 1:
            demand_coords = (demand.loc[i, 'X'], demand.loc[i, 'Y'])
            candidate_coords = (candidate.loc[j, 'X'], candidate.loc[j, 'Y'])
            distance = calculate_distance(demand_coords[0], demand_coords[1], candidate_coords[0], candidate_coords[1])
            if distance <= 1.0:
                nearest_candidates.append(candidate.loc[j, '학교 id'])
            else:
                distance_exceeded_results.append((demand_id, demand_population, candidate.loc[j, '학교 id'], distance))
    results.append((demand_id, demand_population, nearest_candidates))

# 거리 초과 결과 출력
print("거리 초과 결과:")
for result in distance_exceeded_results:
    demand_id, demand_population, nearest_candidate, distance = result
    print(f"수요지 id {demand_id} | 초등학령인구: {demand_population} | 할당된 후보지: {nearest_candidate} | 거리: {distance:.2f} km")

거리 초과 결과:
수요지 id C61 | 초등학령인구: 2 | 할당된 후보지: F49 | 거리: 1.01 km
수요지 id C69 | 초등학령인구: 400 | 할당된 후보지: F48 | 거리: 1.03 km
수요지 id C76 | 초등학령인구: 188 | 할당된 후보지: F37 | 거리: 1.17 km
수요지 id C77 | 초등학령인구: 52 | 할당된 후보지: F48 | 거리: 1.04 km
수요지 id C83 | 초등학령인구: 210 | 할당된 후보지: F33 | 거리: 1.10 km
수요지 id C85 | 초등학령인구: 73 | 할당된 후보지: F2 | 거리: 1.52 km
수요지 id C86 | 초등학령인구: 164 | 할당된 후보지: F33 | 거리: 1.14 km
수요지 id C87 | 초등학령인구: 171 | 할당된 후보지: F33 | 거리: 1.34 km


In [ ]:
import folium

# 수요지와 후보지 위치 정보 추출
demand_locations = demand[['수요지 id', 'Y', 'X']]
candidate_locations = candidate[['학교 id', 'Y', 'X']]

# 지도 초기화
m = folium.Map(location=[demand_locations['Y'].mean(), demand_locations['X'].mean()], zoom_start=12)

# 수요지 위치 마커 추가
for idx, row in demand_locations.iterrows():
    folium.Marker([row['Y'], row['X']], icon=folium.Icon(color='blue'), tooltip=f"Demand {row['수요지 id']}").add_to(m)

# 후보지 위치 마커 추가
for idx, row in candidate_locations.iterrows():
    folium.Marker([row['Y'], row['X']], icon=folium.Icon(color='green'), tooltip=f"Candidate {row['학교 id']}").add_to(m)

# 선택된 후보지와 수요지 간의 선 그리기 및 거리 표시
for i in range(num_demands):
    for j in range(num_candidates):
        if value(assignments[(i, j)]) == 1:
            demand_coords = (demand_locations.loc[i, 'Y'], demand_locations.loc[i, 'X'])
            candidate_coords = (candidate_locations.loc[j, 'Y'], candidate_locations.loc[j, 'X'])
            distance = calculate_distance(demand_coords[1], demand_coords[0], candidate_coords[1], candidate_coords[0])
            assigned_population = demand.loc[i, '초등학령인구']

            folium.PolyLine([demand_coords, candidate_coords], color="red", weight=2.5, opacity=1).add_to(m)

            tooltip_content = f"Demand ID: {demand.loc[i, '수요지 id']}<br>Distance: {distance:.2f} km<br>Assigned Population: {assigned_population}"
            folium.Marker(demand_coords, icon=folium.Icon(color='blue'), tooltip=folium.Tooltip(tooltip_content)).add_to(m)
# 지도 저장
m.save("result_map.html")

In [ ]:
m

In [ ]:
result_list = []

for i in range(num_demands):
    demand_id = demand.loc[i, '수요지 id']
    demand_population = demand.loc[i, '초등학령인구']

    # 모든 후보지에 대한 정보를 리스트에 추가
    for j in range(num_candidates):
        school_id = candidate.loc[j, '학교 id']
        is_assigned = value(assignments[(i, j)]) == 1
        if is_assigned:
            nearest_candidates.append(school_id)
        result_list.append({'수요지 id': demand_id, '학교 id': school_id, '할당 학령인구': demand_population if is_assigned else 0})

# 결과를 저장할 데이터프레임 생성
output_df = pd.DataFrame(result_list)

# 저장된 결과를 출력
print("저장된 결과:")
print(output_df)

# 결과를 CSV 파일로 저장
output_df.to_csv('./output.csv', index=False)

저장된 결과:
      수요지 id 학교 id  할당 학령인구
0         C1    F2        0
1         C1    F3        0
2         C1    F6        0
3         C1    F8        0
4         C1    F9        0
...      ...   ...      ...
13368   C311   F46        0
13369   C311   F47        0
13370   C311   F48        0
13371   C311   F49        0
13372   C311   F50        0

[13373 rows x 3 columns]


In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/주제1/submission.csv")

for i in range(num_demands):
    demand_id = demand.loc[i, '수요지 id']
    demand_population = demand.loc[i, '초등학령인구']

    for j in range(num_candidates):
        school_id = candidate.loc[j, '학교 id']
        assigned = value(assignments[(i, j)]) == 1

        row_idx = submission.index[
            (submission['수요지 id'] == demand_id) & (submission['학교 id'] == school_id)
        ]

        if len(row_idx) > 0:
            submission.loc[row_idx, '할당 학령인구'] = demand_population if assigned else 0

submission.to_csv("submission.csv", index=False, encoding='cp949')

In [ ]:
accident = pd.read_csv('/content/도로교통공단_교통사고다발지역_20230125.csv', encoding='cp949')
accident

,사고지역관리번호,사고연도,사고유형구분,위치코드,사고다발지역시도시군구,사고지역위치명,사고건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,위도,경도,사고다발지역폴리곤정보,데이터기준일자
0,2013060,2012,스쿨존어린이,11110001,서울특별시 종로구1,서울특별시 종로구 혜화동(혜화초교 부근),2,2,0,1,1,0,37.588413,126.999607,"{type:Polygon,coordinates:[[[127.002009,37.590...",2023-01-25
1,2013060,2012,스쿨존어린이,11290001,서울특별시 성북구1,서울특별시 성북구 종암동(숭례초교 부근),2,2,0,0,1,1,37.595505,127.035733,"{type:Polygon,coordinates:[[[127.0381354,37.59...",2023-01-25
2,2013060,2012,스쿨존어린이,11290002,서울특별시 성북구2,서울특별시 성북구 동소문동7가(정덕초교 부근),2,2,0,2,0,0,37.597571,127.014564,"{type:Polygon,coordinates:[[[127.0169658,37.59...",2023-01-25
3,2013060,2012,스쿨존어린이,11290003,서울특별시 성북구3,서울특별시 성북구 석관동(석관초교 부근),3,3,0,2,1,0,37.610429,127.059596,"{type:Polygon,coordinates:[[[127.0619982,37.61...",2023-01-25
4,2013060,2012,스쿨존어린이,11320001,서울특별시 도봉구1,서울특별시 도봉구 쌍문동(쌍문초교 부근),2,2,0,0,2,0,37.648129,127.024430,"{type:Polygon,coordinates:[[[127.0268336,37.65...",2023-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13358,2022084,2021,보행어린이,45140001,전라북도 익산시1,전라북도 익산시 영등동(천일빌라 부근),3,3,0,0,3,0,35.954539,126.973641,"{type:Polygon,coordinates:[[[126.97543771,35.9...",2023-01-25
13359,2022084,2021,보행어린이,46110001,전라남도 목포시1,전라남도 목포시 용해동(목포용해초교 부근),3,3,0,1,2,0,34.817988,126.397695,"{type:Polygon,coordinates:[[[126.39949123,34.8...",2023-01-25
13360,2022084,2021,보행어린이,46110002,전라남도 목포시2,전라남도 목포시 산정동(목포주안교회 부근),3,3,0,0,3,0,34.805209,126.374488,"{type:Polygon,coordinates:[[[126.37628414,34.8...",2023-01-25
13361,2022084,2021,보행어린이,47130001,경상북도 경주시1,경상북도 경주시 용강동(황성초교 부근),4,4,0,1,3,0,35.859953,129.224346,"{type:Polygon,coordinates:[[[129.22614311,35.8...",2023-01-25


In [ ]:
incheon_accident = accident[accident['사고다발지역시도시군구'].str.contains('인천')]

In [ ]:
incheon_accident1 = incheon_accident[incheon_accident['사고다발지역시도시군구'].str.contains('중구|동구')]


for index, row in incheon_accident1.iterrows():
    location = [row['위도'], row['경도']]
    tooltip = f"사고지역: {row['사고지역위치명']}, 사고건수: {row['사고건수']}건"
    folium.Marker(location=location, icon=folium.Icon(color='purple'), tooltip=tooltip).add_to(m)
m

In [ ]:
m.save("result_map.html")